In [92]:
from nlp_common.acts_reader import ActsReader
import regex
import requests
import pandas as pd
from collections import Counter
import numpy as np

In [26]:
reader = ActsReader('../ustawy')
bills = [ act[2] for act in reader.random_n_acts() ]

In [62]:
new_line_re = regex.compile(r'\n+|\s+|\t+')
bills = [new_line_re.sub(' ', bill) for bill in bills]

[' Dz.U. z 2002 r. Nr 12, poz. 110 USTAWA z dnia 6 lipca 2001 r. o usługach detektywistycznych Rozdział 1 Przepisy ogólne Art. 1. 1. Ustawa określa zasady wykonywania działalności gospodarczej w zakresie usług detektywistycznych, prawa i obowiązki detektywów, oraz zasady i tryb nabywania uprawnień do wykonywania usług detektywistycznych. 2. Przepisów ustawy nie stosuje się wobec osób lub instytucji, które na podstawie odrębnych przepisów mogą podejmować, w ramach wykonywania obowiązków zawodowych, działania posiadające charakter czynności, o których mowa w art. 2 ust. 1, lub działania o zbliżonym charakterze. Art. 2. 1. Usługami detektywistycznymi są czynności, polegające na uzyskiwaniu informacji o osobach, przedmiotach i zdarzeniach, realizowane na podstawie umowy zawartej ze zleceniodawcą, w formach i w zakresie niezastrzeżonych dla organów i instytucji państwowych na mocy odrębnych przepisów, a w szczególności: 1) zbieranie informacji w sprawach cywilnych, 2) zbieranie informacji d

### Tagging

In [56]:
tagged_response = requests.post('http://localhost:9200', ' '.join(bills).encode(encoding='utf-8'))
tagged_response

<Response [200]>

In [61]:
tagged_response.text

'Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnone\n\t.\tinterp\tdisamb\nU\tnone\n\tustawa\tbrev:pun\tdisamb\n.\tnone\n\t.\tinterp\tdisamb\nz\tspace\n\tz\tprep:gen:nwok\tdisamb\n2002\tspace\n\t2002\tadj:sg:gen:m3:pos\tdisamb\nr\tspace\n\trok\tbrev:pun\tdisamb\n.\tnone\n\t.\tinterp\tdisamb\n\nNr\tnewline\n\tnumer\tbrev:npun\tdisamb\n12\tspace\n\t12\tnum:pl:nom:m3:rec\tdisamb\n,\tnone\n\t,\tinterp\tdisamb\npoz\tspace\n\tpozycja\tbrev:pun\tdisamb\n.\tnone\n\t.\tinterp\tdisamb\n110\tspace\n\t110\tadj:sg:nom:f:pos\tdisamb\nUSTAWA\tspace\n\tustawa\tsubst:sg:nom:f\tdisamb\nz\tspace\n\tz\tprep:gen:nwok\tdisamb\ndnia\tspace\n\tdzień\tsubst:sg:gen:m3\tdisamb\n6\tspace\n\t6\tadj:sg:gen:m3:pos\tdisamb\nlipca\tspace\n\tlipiec\tsubst:sg:gen:m3\tdisamb\n2001\tspace\n\t2001\tadj:sg:gen:m3:pos\tdisamb\nr\tspace\n\trok\tbrev:pun\tdisamb\n.\tnone\n\t.\tinterp\tdisamb\no\tspace\n\to\tprep:loc\tdisamb\nusługach\tspace\n\tusługa\tsubst:pl:loc:f\tdisamb\ndetektywistycznych\tspace\n\tdetektywistyczny\tadj:pl:lo

### Bigrams

In [116]:
def create_unigrams(tagged_corpus):
    splitted = [ l for l in tagged_corpus.split('\n') if l != '']
    unigrams = []
    for i in range(0, len(splitted), 2):
        if splitted[i].startswith('\t') or not splitted[i+1].startswith('\t'):
            raise Exception("Wrong assumption")
            
        tagging =  splitted[i+1].split('\t')
        tags = tagging[2].split(':')
        unigrams.append(f'{tagging[1].lower()}:{tags[0]}')
    
    return unigrams
    
unigrams = create_unigrams(tagged_response.text)

In [117]:
def create_bigrams(unigrams):
    return [(unigrams[i], unigrams[i+1]) for i in range(len(unigrams)-1)]
    
bigrams = create_bigrams(unigrams)

In [118]:
bigrams

[('dziennik:brev', '.:interp'),
 ('.:interp', 'ustawa:brev'),
 ('ustawa:brev', '.:interp'),
 ('.:interp', 'z:prep'),
 ('z:prep', '2002:adj'),
 ('2002:adj', 'rok:brev'),
 ('rok:brev', '.:interp'),
 ('.:interp', 'numer:brev'),
 ('numer:brev', '12:num'),
 ('12:num', ',:interp'),
 (',:interp', 'pozycja:brev'),
 ('pozycja:brev', '.:interp'),
 ('.:interp', '110:adj'),
 ('110:adj', 'ustawa:subst'),
 ('ustawa:subst', 'z:prep'),
 ('z:prep', 'dzień:subst'),
 ('dzień:subst', '6:adj'),
 ('6:adj', 'lipiec:subst'),
 ('lipiec:subst', '2001:adj'),
 ('2001:adj', 'rok:brev'),
 ('rok:brev', '.:interp'),
 ('.:interp', 'o:prep'),
 ('o:prep', 'usługa:subst'),
 ('usługa:subst', 'detektywistyczny:adj'),
 ('detektywistyczny:adj', 'rozdział:subst'),
 ('rozdział:subst', '1:adj'),
 ('1:adj', 'przepis:subst'),
 ('przepis:subst', 'ogólny:adj'),
 ('ogólny:adj', 'art:subst'),
 ('art:subst', '.:interp'),
 ('.:interp', '1:adj'),
 ('1:adj', '.:interp'),
 ('.:interp', '1:adj'),
 ('1:adj', '.:interp'),
 ('.:interp', 'usta

### LLR

In [122]:
def compute_global_counter(ctrs): 
    global_counter = Counter()
    for ctr in ctrs:
        global_counter += ctr
    return global_counter

def probabilities(ngrams, filter_fn=None):
    ctrs = [Counter(doc) for doc in ngrams]
    global_counter = compute_global_counter(ctrs)
    l = len(global_counter)
    probs = { k: global_counter[k]/l for k in global_counter 
            if filter_fn == None or filter_fn(k, global_counter[k])}
    global_counter = { k: global_counter[k] for k in global_counter
            if filter_fn == None or filter_fn(k, global_counter[k]) }
    return probs, global_counter

def probabilities_flat(counter_ngrams, map_key_fn):
    ctr = Counter()
    for k in counter_ngrams:
        mk = map_key_fn(k)
        ctr[mk] += counter_ngrams[k]
    l = len(ctr)
    probs = {
        k: ctr[k]/l for k in ctr
    }
    return probs, ctr

def pmi_df_bigram(bill_docs_bigrams, filter_fn=None):
    pm, pmctr = probabilities(bill_docs_bigrams, filter_fn)
    pa, pactr = probabilities_flat(pmctr, lambda x: x[0])
    pb, pbctr = probabilities_flat(pmctr, lambda x: x[1])

    df = pd.DataFrame(data=[(k,) for k in pm.keys()], columns = ["bigram"])
    df["p(a)"] = df["bigram"].apply(lambda x: pa[x[0]])
    df["#a"] = df["bigram"].apply(lambda x: pactr[x[0]])
    df["p(b)"] = df["bigram"].apply(lambda x: pb[x[1]])
    df["#b"] = df["bigram"].apply(lambda x: pbctr[x[1]])
    df["p(a,b)"] = df["bigram"].apply(lambda x: pm[x])
    df["#ab"] = df["bigram"].apply(lambda x: pmctr[x])
    df["pmi"] = np.log(df["p(a,b)"]/ (df["p(a)"]*df["p(b)"]))
    return df#.sort_values(["pmi"], ascending=[0])

def H(a):
    a = np.array(a)
    N = a.sum()
    return np.sum(a/N * np.log(a/N + (a==0)))
                                  
def llr(pmi_df):
    def apply_llr(k):
        return 2*np.sum(k)*(H(k) - H(k.sum(axis=0)) - H(k.sum(axis=1)))
        
    df = pmi_df.copy()
    sum_all = pmi_df["#ab"].sum()
    df["k11"] = pmi_df["#ab"]
    df["k12"] = pmi_df["#a"] - df["k11"]
    df["k21"] = pmi_df["#b"] - df["k11"] 
    df["k22"] = sum_all - (pmi_df["#a"] + pmi_df["#b"] - pmi_df["#ab"])
    df["k"] = df[["k11", "k12","k21", "k22"]].values.tolist()
    df["k"] = df["k"].apply(lambda k: np.array(k).reshape((2,2)))
    df["llr"] = df["k"].apply(apply_llr)
    pmi_df["llr"] = df["llr"]
    return pmi_df.sort_values(["llr"], ascending=[0])

In [123]:
bigram_regex = regex.compile('\p{L}+:\p{L}+')
llrdf = llr(pmi_df_bigram([bigrams], lambda k,_: bigram_regex.match(k[0]) and bigram_regex.match(k[1])))

In [124]:
llrdf.head(30)

,bigram,p(a),#a,p(b),#b,"p(a,b)",#ab,pmi,llr
48,"(który:adj, mowa:subst)",0.171927,512,0.094970,287,0.014390,287,-0.126377,2535.878680
47,"(o:prep, który:adj)",0.223304,665,0.125083,378,0.014440,288,-0.659773,1924.826766
49,"(mowa:subst, w:prep)",0.096373,287,0.468564,1416,0.014289,285,-1.150637,1773.763276
50,"(w:prep, artykuł:brev)",0.765279,2279,0.177366,536,0.019353,386,-1.947841,1436.103994
448,"(minister:subst, właściwy:adj)",0.063465,189,0.066512,201,0.007220,144,0.536709,1377.009237
149,"(w:prep, ustęp:brev)",0.765279,2279,0.120450,364,0.014089,281,-1.878344,1100.956918
441,"(właściwy:adj, do:prep)",0.069510,207,0.204169,617,0.007771,155,-0.602215,1013.194280
450,"(sprawa:subst, wewnętrzny:adj)",0.087643,261,0.037392,113,0.005214,104,0.464430,971.482811
4726,"(dodawać:fin, się:qub)",0.041974,125,0.196889,595,0.006167,123,-0.292743,970.038055
1664,"(sejmik:subst, województwo:subst)",0.036602,109,0.128061,387,0.005315,106,0.125614,925.939382


## Partition